In [1]:
#importing library for webscrapping.for this purpose we need request and urllib.
#requests is used for requesting a data from client side.
import requests
import urllib
# BeautifulSoup is used as a replacement for regular expression for html.
from bs4 import BeautifulSoup
# importing nltk tools for tokenizing word and sentence.
from nltk.tokenize import word_tokenize,sent_tokenize
from nltk.corpus import stopwords
from string import punctuation

In [2]:
# loading the data from thehindu.com
# returing text and its heading.
def get_text_from_hindu(url):
    try:
        page = urllib.request.urlopen(url).read().decode('utf8')
    except:
        return (None,None)
    
    soup = BeautifulSoup(page)
    
    if soup is None:
        return (None,None)
    
    text = ''
    if soup.find_all('p') is not None:
        text = ''.join(map(lambda x:x.text,soup.find_all('p')[1:-4]))
        
    return text,soup.title.text

In [3]:
article = get_text_from_hindu('http://www.thehindu.com/sport/cricket/bizarre-decisions-proved-costly-for-tn-ranji-team/article21119357.ece')
article

C:\Users\Indresh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Indresh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


('The underwhelming Tamil Nadu, lacking consistency and cutting edge, did not deserve to go through to the knock-out stages of the Ranji Trophy.The side’s tactics and the selection of its eleven appeared bizarre on occasions. The different arms of the team did not move in cohesion.The signs were ominous when Tamil Nadu crawled to 112 for two in 54 overs on day three after conceding the lead and needing to force the pace against Andhra in the season-opener. The defensive mind-set became pronounced as the season progressed, in several decisive moments of the league phase.Despite gaining the lead against Mumbai after an early stumble with B. Indrajith and V. Yo Mahesh notching up centuries, Tamil Nadu could not build on the momentum gained.The side, inexplicably, conceded the lead to Odisha after notching up 530 for eight. The bowling unit stood exposed. Over-dependence on left-arm spinner Rahil Shah — effective on turners and ordinary on other tracks — did not help the side’s cause. His 

In [4]:
class FrequencyCounter():
    
    def __init__(self,min_cut=0.1,max_cut=0.9):
        self.min_cut = min_cut
        self.max_cut = max_cut
        
        self._stopwords = set(stopwords.words('english')+list(punctuation)+['.',',',"'", '“', '”','’', '‘'])
        
    def _compute_frequency(self,word_sent):
        freq_dict = {}
        
        for sent in word_sent:
            for word in sent:
                if word not in self._stopwords:
                    c = freq_dict.get(word,0)
                    freq_dict[word]=c+1
    
        m = float(max(freq_dict.values()))
        
        print(freq_dict)
        #eliminating unnecessary word using max_cut and min_cut
        for w in list(freq_dict.keys()):
            freq_dict[w] = freq_dict[w]/m
            if freq_dict[w]<self.min_cut or freq_dict[w]>self.max_cut:
                del freq_dict[w]
                
        return freq_dict
    
    def extractFeatures(self,article,n):
        text = article[0]
        sentence = sent_tokenize(text)
        word_sent = [word_tokenize(s.lower()) for s in sentence]
        self._freq = self._compute_frequency(word_sent)
        #print(self._freq)
        list1 = sorted(self._freq.keys(),key= lambda x:self._freq[x],reverse=True)
        return list1[:n]
    
    def summarize(self,text,n):
        sents = sent_tokenize(text)
        
        assert n <= len(sents)
        # assert is a way of making sure a condition holds true, else an exception is thrown. Used to do 
        # sanity checks like making sure the summary is shorter than the original article.
        
        word_sent = [word_tokenize(s.lower()) for s in sents]
        freq = self._compute_frequency(word_sent)
        #print("\n\nfrequency->\n",freq,"\n\n")
        ranking = {}
        for i,sent in enumerate(word_sent):
            ranking[i]=0
            for word in sent:
                if word in freq:
                    ranking[i]+=freq[word]
                    
        sort_index = sorted(ranking.keys(),key=lambda x:ranking[x], reverse =True)
        top_n = [sents[j] for j in sort_index[:n]]
        return top_n
    
    

In [5]:
fs = FrequencyCounter()
#fs.extractFeatures(article,2)
fs.summarize(article[0],2)
fs.extractFeatures(article,25)

{'underwhelming': 1, 'tamil': 5, 'nadu': 6, 'lacking': 1, 'consistency': 1, 'cutting': 1, 'edge': 1, 'deserve': 1, 'go': 1, 'knock-out': 1, 'stages': 1, 'ranji': 2, 'trophy.the': 1, 'side': 6, 'tactics': 2, 'selection': 1, 'eleven': 3, 'appeared': 1, 'bizarre': 1, 'occasions': 1, 'different': 1, 'arms': 1, 'team': 1, 'move': 1, 'cohesion.the': 1, 'signs': 1, 'ominous': 1, 'crawled': 1, '112': 1, 'two': 2, '54': 1, 'overs': 1, 'day': 1, 'three': 2, 'conceding': 1, 'lead': 3, 'needing': 1, 'force': 1, 'pace': 1, 'andhra': 2, 'season-opener': 1, 'defensive': 1, 'mind-set': 1, 'became': 1, 'pronounced': 1, 'season': 1, 'progressed': 1, 'several': 1, 'decisive': 1, 'moments': 2, 'league': 1, 'phase.despite': 1, 'gaining': 1, 'mumbai': 1, 'early': 1, 'stumble': 1, 'b.': 2, 'indrajith': 2, 'v.': 1, 'yo': 2, 'mahesh': 2, 'notching': 2, 'centuries': 1, 'could': 1, 'build': 1, 'momentum': 1, 'gained.the': 1, 'inexplicably': 1, 'conceded': 1, 'odisha': 1, '530': 1, 'eight': 1, 'bowling': 2, 'unit

['tamil',
 'bowlers',
 'eleven',
 'lead',
 'ranji',
 'tactics',
 'two',
 'three',
 'andhra',
 'moments',
 'b.',
 'indrajith',
 'yo',
 'mahesh',
 'notching',
 'bowling',
 '—',
 'turners',
 'tracks',
 'came',
 'average',
 'many',
 'all-rounders',
 'make',
 'ability']

In [6]:
## for cricket news 
def scrapeSite(url,scraperFunction,magicFrag = 'cricket',token = None):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    soup1 = BeautifulSoup(response)
    
    urldict = {}
    
    numerror = 0
    for a in soup1.findAll('a'):
        try:
            url=a.get('href')
#            if ((url not in urldict) and ((magicFrag is not None and magicFrag in url) or magicFrag is None)):
            if( (url not in urldict) and ((magicFrag is not None and magicFrag in url) or magicFrag is None)):
                body = scraperFunction(url)
#                print(body)                    
                if body and len(body)>0:
                    urldict[url] = body
#                print(url)
                
        except:
            numerror +=1
    return urldict

In [7]:
urldict = scrapeSite("http://www.thehindu.com/sport/cricket/", get_text_from_hindu,'cricket')


C:\Users\Indresh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Indresh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [8]:
urldict

{'http://www.thehindu.com/sport/cricket/': ("\nMitchell Marsh and Travis Head to captain the four-day and one-day teams respectively in September\n\nChennai: Mahendra Singh Dhoni loves number 7, inscribed on his jersey for India and Chennai Super Kings. He considers it his good luck charm, \nThe toss that brings in an element of intrigue at the start of a Test will remain in force although former captains and players had wanted it to be \nIn this edition of the IPL, Chennai Super Kings skipper MS Dhoni had the highest average and strike rate among batsmen. On Sunday, Dhoni emphasised  \n\n\nBanned Warner, Bancroft to play in Northern Territory in July\n  \n\n\nFleming — CSK’s man for all seasons\n  \n\n\nSuper Kings take ‘core group’ definition to a new level\n \nWatson was uncertain about his future after his stint with RCB last season\n \n\n\nTop Indian cricketers who have represented English counties\n  \n\n\nWhat's the win ratio of your favourite IPL team?\n  \n\n\nRabada ruled out

In [9]:
# for economy news
def scrapeSite1(url,scraperFunction,magicFrag = 'Economy',token = None):
    request = urllib.request.Request(url)
    response = urllib.request.urlopen(request)
    soup1 = BeautifulSoup(response)
    
    urldict = {}
    
    numerror = 0
    for a in soup1.findAll('a'):
        try:
            url=a.get('href')
#            if ((url not in urldict) and ((magicFrag is not None and magicFrag in url) or magicFrag is None)):
            if( (url not in urldict) and ((magicFrag is not None and magicFrag in url) or magicFrag is None)):
                body = scraperFunction(url)
#                print(body)                    
                if body and len(body)>0:
                    urldict[url] = body
#                print(url)
                
        except:
            numerror +=1
    return urldict

In [10]:
ulldict2 = scrapeSite1("http://www.thehindu.com/business/Economy/", get_text_from_hindu,'Economy')

C:\Users\Indresh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Indresh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [11]:
# contains the conclusion of every news as cricket or ecocnomy.
articleSummaries = {}


for articleUrl in urldict:
    if urldict[articleUrl][1] is not None:
        if len(urldict[articleUrl][0]) > 0:
            fs = FrequencyCounter()
            summary = fs.extractFeatures(urldict[articleUrl],25)
            articleSummaries[articleUrl] = {'feature-vector': summary,
                                           'label': 'cricket'}
for articleUrl in ulldict2:
    if ulldict2[articleUrl][1] is not None:
        if len(ulldict2[articleUrl][0]) > 0:
            fs = FrequencyCounter()
            summary = fs.extractFeatures(ulldict2[articleUrl],25)
            articleSummaries[articleUrl] = {'feature-vector': summary,
                                           'label': 'economy'}

{'mitchell': 1, 'marsh': 1, 'travis': 1, 'head': 1, 'captain': 1, 'four-day': 1, 'one-day': 1, 'teams': 1, 'respectively': 1, 'september': 1, 'chennai': 3, 'mahendra': 1, 'singh': 1, 'dhoni': 3, 'loves': 1, 'number': 1, '7': 1, 'inscribed': 1, 'jersey': 1, 'india': 1, 'super': 3, 'kings': 3, 'considers': 1, 'good': 1, 'luck': 1, 'charm': 1, 'toss': 1, 'brings': 1, 'element': 1, 'intrigue': 1, 'start': 1, 'test': 1, 'remain': 1, 'force': 1, 'although': 1, 'former': 1, 'captains': 1, 'players': 1, 'wanted': 1, 'edition': 1, 'ipl': 3, 'skipper': 1, 'ms': 1, 'highest': 1, 'average': 1, 'strike': 1, 'rate': 1, 'among': 1, 'batsmen': 1, 'sunday': 1, 'emphasised': 1, 'banned': 1, 'warner': 2, 'bancroft': 1, 'play': 1, 'northern': 1, 'territory': 1, 'july': 1, 'fleming': 1, '—': 1, 'csk': 1, 'man': 1, 'seasons': 1, 'take': 1, 'core': 1, 'group': 1, 'definition': 1, 'new': 1, 'level': 1, 'watson': 1, 'uncertain': 1, 'future': 2, 'stint': 1, 'rcb': 1, 'last': 1, 'season': 1, 'top': 1, 'indian': 

{'shane': 2, 'watson': 5, 'single-handedly': 1, 'guided': 1, 'chennai': 2, 'super': 3, 'kings': 2, 'third': 1, 'ipl': 4, 'title': 2, 'unbeaten': 1, 'knock': 1, '117': 1, 'sunrisers': 2, 'hyderabad': 1, 'summit': 1, 'clash': 2, 'losing': 1, 'captain': 4, 'kane': 1, 'williamson': 7, 'left': 1, 'admire': 1, 'burly': 1, 'australian': 1, 'unbelievable': 2, 'innings.the': 1, '36-year-old': 1, 'smashed': 1, '11': 1, 'fours': 1, 'eight': 1, 'sixes': 1, 'en': 1, 'route': 1, 'second': 1, 'hundred': 1, 'season': 3, 'csk': 8, 'chased': 1, 'target': 2, '179': 1, '18.3': 1, 'overs': 5, 'last': 2, 'night.': 2, 'yes': 1, 'think': 1, 'brilliant': 2, 'innings': 4, 'anyone': 2, 'turns': 1, '100-plus': 1, 'final': 2, 'obviously': 1, 'significant': 1, 'effort': 2, 'great': 3, 'contribution': 1, 'side': 6, 'difficult': 2, 'stop': 1, 'said': 7, 'suffered': 1, '8-wicket': 1, 'loss': 1, '2018': 2, 'vs': 1, 'srh': 2, 'make': 1, 'fairytale': 1, 'return': 1, 'large': 1, 'part': 1, 'game': 4, 'lot': 1, 'good': 4, 

{'mitchell': 1, 'marsh': 1, 'travis': 1, 'head': 1, 'captain': 1, 'four-day': 1, 'one-day': 1, 'teams': 1, 'respectively': 1, 'september': 1, 'chennai': 3, 'mahendra': 1, 'singh': 1, 'dhoni': 3, 'loves': 1, 'number': 1, '7': 1, 'inscribed': 1, 'jersey': 1, 'india': 1, 'super': 3, 'kings': 3, 'considers': 1, 'good': 1, 'luck': 1, 'charm': 1, 'toss': 1, 'brings': 1, 'element': 1, 'intrigue': 1, 'start': 1, 'test': 1, 'remain': 1, 'force': 1, 'although': 1, 'former': 1, 'captains': 1, 'players': 1, 'wanted': 1, 'edition': 1, 'ipl': 3, 'skipper': 1, 'ms': 1, 'highest': 1, 'average': 1, 'strike': 1, 'rate': 1, 'among': 1, 'batsmen': 1, 'sunday': 1, 'emphasised': 1, 'banned': 1, 'warner': 2, 'bancroft': 1, 'play': 1, 'northern': 1, 'territory': 1, 'july': 1, 'fleming': 1, '—': 1, 'csk': 1, 'man': 1, 'seasons': 1, 'take': 1, 'core': 1, 'group': 1, 'definition': 1, 'new': 1, 'level': 1, 'watson': 1, 'uncertain': 1, 'future': 2, 'stint': 1, 'rcb': 1, 'last': 1, 'season': 1, 'top': 1, 'indian': 

{'ranjan': 1, 'de': 1, 'silva': 2, 'local': 1, 'politician': 1, 'shot': 1, 'dead': 1, 'near': 1, 'colombo': 1, 'thursday': 1, 'hours': 1, 'team': 1, 'due': 1, 'leave': 1, 'three-match': 1, 'test': 1, 'series': 1, 'west': 1, 'indies.': 1, 'replacement': 1, 'dhananjaya': 2, 'board': 1, 'official': 1, 'told': 1, 'afp': 1, 'instead': 1, '17-member': 1, 'squad': 2, '16.': 1, 'sri': 1, 'lanka': 2, 'cricket': 1, 'said': 2, 'would': 1, 'support': 1, '26-year-old': 1, 'time': 1, 'sorrow': 1, 'grief': 1, '.police': 1, 'investigation': 1, 'underway': 1, 'arrests': 1, 'made.sri': 1, 'also': 1, 'miss': 1, 'opener': 1, 'dimuth': 1, 'karunaratne': 1, 'fractured': 1, 'finger': 1, 'batting': 1, 'nets': 1, 'month': 1, 'prepared': 1, 'domestic': 1, 'tournament.the': 1, 'revised': 1, 'dinesh': 1, 'chandimal': 1, 'capt': 1, 'mahela': 1, 'udawatte': 1, 'kusal': 2, 'mendis': 1, 'janith': 1, 'perera': 2, 'roshen': 1, 'angelo': 1, 'mathews': 1, 'niroshan': 1, 'dickwella': 1, 'rangana': 1, 'herath': 1, 'dilruwa

{'petrol': 4, 'price': 3, 'sunday': 1, 'touched': 1, 'record': 1, 'high': 1, '₹76.24': 1, 'per': 1, 'litre': 4, 'diesel': 4, 'stood': 2, '₹67.57': 1, 'national': 1, 'capital': 1, 'oil': 3, 'firms': 1, 'raised': 3, 'prices': 6, 'seventh': 1, 'day': 2, 'row.the': 1, 'went': 1, '33': 1, 'paise': 3, '26': 1, 'delhi': 2, 'according': 1, 'notification': 1, 'issued': 1, 'state-owned': 1, 'firms.rates': 1, 'vary': 1, 'across': 1, 'states': 1, 'depending': 1, 'local': 1, 'sales': 1, 'tax': 1, 'vat': 1, 'cheapest': 1, 'among': 1, 'metros': 1, 'state': 1, 'capitals': 1, '₹78.91': 1, 'kolkata': 2, '₹84.07': 1, 'mumbai': 2, '₹79.13': 1, 'chennai': 1, '₹70.12': 1, '₹71.94': 1, '₹71.32': 1, 'chennai.fuel': 1, 'revised': 1, 'upward': 1, 'every': 1, 'since': 2, 'may': 2, '14': 1, 'daily': 1, 'revisions': 1, 'place': 1, 'almost': 2, 'year': 1, 'companies': 1, 'halted': 1, 'practice': 1, 'three': 1, 'weeks': 1, 'april': 1, '24': 1, 'fuels': 1, '13': 1, '₹1.61': 1, '₹1.64': 1, 'past': 1, 'week.during': 1,

In [12]:
for i in articleSummaries.keys():
    print('\n',i,'\n',articleSummaries[i],'\n')


 http://www.thehindu.com/sport/cricket/ 
 {'feature-vector': ['warner', 'future', 'team', 'mitchell', 'marsh', 'travis', 'head', 'captain', 'four-day', 'one-day', 'teams', 'respectively', 'september', 'mahendra', 'singh', 'loves', 'number', '7', 'inscribed', 'jersey', 'india', 'considers', 'good', 'luck', 'charm'], 'label': 'cricket'} 


 http://www.thehindu.com/sport/cricket/star-studded-australia-a-squads-named-for-india-tour/article24031556.ece 
 {'feature-vector': ['tour', 'squad', 'travis', 'alex', 'players', 'marsh', 'four-day', 'head', 'mitch', 'india', 'one-day', 'test', 'experience', 'group', 'carey', 'ashton', 'agar', 'peter', 'handscomb', 'usman', 'khawaja', 'matthew', 'renshaw', 'chris', 'opportunity'], 'label': 'cricket'} 


 http://www.thehindu.com/sport/cricket/icc-meeting-the-flip-of-coin-stays/article24027376.ece 
 {'feature-vector': ['toss', 'visiting', 'cricket', 'game', 'also', 'spirit', 'code', 'conduct', 'support', 'new', 'offence', 'offensive', 'away', 'english'

In [13]:
def gethindustantimestext(url):
    page = urllib.request.urlopen(url).read().decode('utf8').encode('cp850','replace').decode('cp850')
    soup = BeautifulSoup(page)
    divs = soup.findAll('div',{'class':'story-details'})
    text = ''.join(map(lambda x:x.text,divs))
    return text,soup.find('title').text


In [23]:
#hturl = "http://www.hindustantimes.com/india-vs-sri-lanka-2017/ms-dhoni-still-pillar-of-indian-cricket-team-after-defiant-knock-in-dharamsala/story-PpwHKt9ruEEI5ghMvtun0I.html"
#hturl = "https://www.hindustantimes.com/cricket/dale-steyn-there-are-great-players-and-then-there-s-ab-de-villiers/story-kcDFnbwxtgdHPlivGqVYmL.html"
#testarticle = gethindustantimestext(hturl)
#fs1 = FrequencyCounter()
#hindusumm = fs1.extractFeatures(testarticle,25)


In [26]:

"""simmilarity = {}
for articleurl in articleSummaries:
    onearticlesumm = articleSummaries[articleurl]['feature-vector']
    simmilarity[articleurl]=len(set(hindusumm).intersection(set(onearticlesumm)))
"""

"simmilarity = {}\nfor articleurl in articleSummaries:\n    onearticlesumm = articleSummaries[articleurl]['feature-vector']\n    simmilarity[articleurl]=len(set(hindusumm).intersection(set(onearticlesumm)))\n"

In [27]:
simmilarity

{'http://www.thehindu.com/business/Economy/all-macroeconomic-parameters-improved-in-last-four-years-niti-aayog/article24005211.ece': 2,
 'http://www.thehindu.com/business/Economy/an-inch-forward-but-miles-to-go/article23874501.ece': 4,
 'http://www.thehindu.com/business/Economy/banking-services-impacted-as-psu-staff-go-on-two-day-strike/article24030928.ece': 2,
 'http://www.thehindu.com/business/Economy/chinese-state-run-bank-launches-india-dedicated-investment-fund/article23879793.ece': 2,
 'http://www.thehindu.com/business/Economy/coc-is-not-proactive-enough/article23993633.ece': 1,
 'http://www.thehindu.com/business/Economy/considering-steps-to-keep-fuel-prices-in-check-dharmendra-pradhan/article23951601.ece': 1,
 'http://www.thehindu.com/business/Economy/economic-indicators-better-niti/article24007774.ece': 0,
 'http://www.thehindu.com/business/Economy/for-nri-flows-may-2018-is-the-best-in-nearly-five-years/article24027135.ece': 0,
 'http://www.thehindu.com/business/Economy/forex-r

In [28]:
from collections import defaultdict
labels = defaultdict(int)  
knn = sorted(simmilarity.keys(),key = lambda x:simmilarity[x],reverse = True)[:5]
print(knn)
for i in knn:
    labels[articleSummaries[i]['label']]+=1
print(labels)

['http://www.thehindu.com/business/Economy/why-are-crude-oil-prices-going-up/article23937146.ece', 'http://www.thehindu.com/business/Economy/india-resists-lobbying-by-us-payment-firms-to-ease-local-data-storage-rules/article24030702.ece', 'http://www.thehindu.com/business/Economy/pakistan-seeks-economic-lifeline-with-fresh-china-loans/article24002329.ece', 'http://www.thehindu.com/business/Economy/india-sixth-wealthiest-country-with-total-wealth-of-8230-billion/article23942050.ece', 'http://www.thehindu.com/business/Economy/an-inch-forward-but-miles-to-go/article23874501.ece']
defaultdict(<class 'int'>, {'economy': 5})


In [32]:
# Predicting the economy class
url_buss = "http://www.thehindu.com/business/Economy/need-to-clarify-development-within-wto-us/article21461527.ece"
testarticle = get_text_from_hindu(url_buss)
fs1 = FrequencyCounter()
hindusumm = fs1.extractFeatures(testarticle,25)

simmilarity = {}
for articleurl in articleSummaries:
    onearticlesumm = articleSummaries[articleurl]['feature-vector']
    simmilarity[articleurl]=len(set(hindusumm).intersection(set(onearticlesumm)))
    
labels1 = defaultdict(int)  
knn = sorted(simmilarity.keys(),key = lambda x:simmilarity[x],reverse = True)[:5]
print(knn)
for i in knn:
    labels1[articleSummaries[i]['label']]+=1
print("\n --------------------------\n Predictin is:-\n")    
print(labels1)

{'veiled': 1, 'attack': 1, 'countries': 4, 'including': 2, 'india': 9, 'china': 3, 'batting': 1, 'improvement': 1, 'trading': 2, 'prospects': 2, 'developing': 7, 'world': 3, 'u.s.': 6, 'said': 10, 'need': 3, 'clarify': 2, 'understanding': 2, 'development': 6, 'within': 2, 'context': 1, 'trade': 8, 'organisation': 3, 'wto': 17, '.in': 1, 'opening': 1, 'plenary': 2, 'statement': 2, 'meeting': 1, 'highest': 1, 'decision-making': 1, 'body': 2, 'held': 1, 'representative': 1, 'ustr': 2, 'robert': 1, 'lighthizer': 5, 'sustain': 1, 'situation': 1, 'new': 4, 'rules': 4, 'apply': 1, 'others': 1, 'given': 1, 'pass': 1, 'name': 1, 'self-proclaimed': 1, 'status.': 1, 'comes': 1, 'backdrop': 1, 'commerce': 1, 'minister': 2, 'suresh': 1, 'prabhu': 3, 'saying': 1, 'session': 1, 'ongoing': 1, 'ministerial': 1, 'conference': 1, 'expansion': 2, 'global': 2, 'vision': 1, 'outcome': 1, 'must': 1, 'development.mr': 1, 'remarks': 1, 'something': 1, 'wrong': 1, 'view': 1, 'five': 1, 'six': 1, 'richest': 1, '

C:\Users\Indresh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Indresh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


In [34]:
# Lets predict the cricket class
url_cricket = "http://www.thehindu.com/sport/cricket/star-studded-australia-a-squads-named-for-india-tour/article24031556.ece"
testarticle = get_text_from_hindu(url_cricket)
fs1 = FrequencyCounter()
hindusumm = fs1.extractFeatures(testarticle,25)

simmilarity = {}
for articleurl in articleSummaries:
    onearticlesumm = articleSummaries[articleurl]['feature-vector']
    simmilarity[articleurl]=len(set(hindusumm).intersection(set(onearticlesumm)))
    
labels1 = defaultdict(int)  
knn = sorted(simmilarity.keys(),key = lambda x:simmilarity[x],reverse = True)[:5]
print(knn)
for i in knn:
    labels1[articleSummaries[i]['label']]+=1
print("\n --------------------------------\n Prediction is:-\n")
print(labels1)

{'all-rounder': 1, 'mitchell': 2, 'marsh': 4, 'wednesday': 1, 'named': 1, 'skipper': 1, 'star-studded': 1, 'australia': 7, 'squad': 5, 'four-day': 4, 'games': 2, 'tour': 6, 'india': 3, 'later': 1, 'year': 1, 'travis': 5, 'head': 4, 'lead': 2, 'one-day': 3, 'side.the': 1, 'side': 1, 'compete': 1, 'tri-series': 1, 'south': 2, 'africa': 2, 'vijayawada': 1, 'august': 1, 'ahead': 1, 'september.marsh': 1, 'whose': 1, 'test': 3, 'vice': 1, 'captaincy': 2, 'prospects': 1, 'got': 2, 'boost': 1, 'leads': 1, '14-man': 1, 'packed': 1, 'international': 2, 'experience': 3, 'group': 3, 'featuring': 1, 'alex': 5, 'carey': 3, 'ashton': 3, 'agar': 3, 'peter': 3, 'handscomb': 3, 'jon': 2, 'holland': 2, 'usman': 3, 'khawaja': 3, 'joel': 2, 'paris': 2, 'matthew': 3, 'renshaw': 3, 'chris': 3, 'tremain.test': 1, 'legend': 1, 'adam': 1, 'gilchrist': 1, 'backed': 1, 'tim': 1, 'paine': 1, 'vice-captain': 2, '26-year-old': 1, 'perfect': 1, 'opportunity': 3, 'display': 1, 'leadership': 1, 'qualities': 1, 'two': 1

C:\Users\Indresh\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\Indresh\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
